## Delvecchio Giovanni Pio mat. 0001037185

###Deep Learning Exam

The task consists in building a pipeline, comprehending a Neural Network using Tensorflow, which is capable of classifying a corpus of texts with respect to one between the emotions defined by Paul Ekman (Anger, Disgust, Fear, Joy, Sadness and Surprise).

In order to do this, the following steps have been coded:
- Dataset Cleaning
- Encoding of the target classes
- Segmentation of the sentences
- Tokenization
- Construction of the model
- Training of the model
- Plotting of the results 

### Setting up the environment for training the model on GPU

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Installation of an external library for text analisys and representation

source:
https://github.com/fucaja/ekphrasis

In [ ]:
!pip install git+https://github.com/fucaja/ekphrasis.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/fucaja/ekphrasis.git to /tmp/pip-req-build-15vkm4ua
  Running command git clone -q https://github.com/fucaja/ekphrasis.git /tmp/pip-req-build-15vkm4ua
     |████████████████████████████████| 45 kB 3.7 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-py3-none-any.whl size=614879 sha256=a0d08e119598003b1b2b440bcabf3090d24a6601a26c726bbd7e7a7ddc13151d
  Stored in directory: /tmp/pip-ephem-wheel-cache-6yq7v93x/wheels/ba/6d/16/386c2ee4778a28420fcfea1a1aadf7bac121198ace6cc5d354
Successfully built ekphrasis


## Installation of the library needed for emoji representation

source:
https://pypi.org/project/demoji/

In [ ]:
!pip install demoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.4 MB/s 


## Installation of the library needed in order to build the model

The choice has been RoBERTa base.
This model features more parameters than BERT and, in fine tuning tasks like this one, it has shown good results at different levels of complexity of the remaining part of the network (the one that has to be trained).

Sources:
- https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9716923
- https://www.researchgate.net/publication/333740389_A_Comparison_of_Word-Embeddings_in_Emotion_Detection_from_Text_using_BiLSTM_CNN_and_Self-Attention#pf6

In particular the best model for this task has proved to be a variation of the one proposed in the second linked paper (Polignano, Basile, Gemmis, Semeraro)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 34.9 MB/s 
     |████████████████████████████████| 596 kB 57.5 MB/s 
     |████████████████████████████████| 101 kB 11.5 MB/s 
     |████████████████████████████████| 6.6 MB 65.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Installation of the library needed in order to use a self attention layer

In [ ]:
!pip install keras-self-attention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18912 sha256=d26ac3cd19107dfe9b4bf5fd5652baad9f148101ddf11d66c3127c907fa1dbec
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-self-attention


## Installation of tensorflow-addons, needed to import the F1 Score supported by keras

In [ ]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 37.2 MB/s 


## Imports from the libraries

In [ ]:
# data structures handling
import numpy as np
import pandas as pd

# preprocessing
import demoji
import ekphrasis
from demoji import replace_with_desc
from ekphrasis.classes.spellcorrect import SpellCorrector
from ekphrasis.classes.segmenter import Segmenter
from ekphrasis.classes.preprocessor import TextPreProcessor
import nltk
from sklearn.preprocessing import OneHotEncoder
import re

#model building
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaModel
from keras import backend as K
from keras import regularizers
from tensorflow.keras.layers import LSTM, Bidirectional, Dropout, Dense, GlobalMaxPool1D
from tensorflow.keras.layers import Convolution1D, MaxPooling1D, Concatenate
from tensorflow_addons.metrics import F1Score
from keras_self_attention import SeqSelfAttention

#testing 
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

print("all loaded")

all loaded


## Loading of the dataset

In order to load the dataset it's necessary to upload it on colab and run the cell, the default directory is <code>/content/</code>

In [ ]:
train_set = pd.read_csv("/content/train_ekmann.csv")
val_set = pd.read_csv("/content/val_ekmann.csv")
test_set = pd.read_csv("/content/test_ekmann.csv")

print(train_set.head())
print(val_set.head())
print(test_set.head())

                                                Text  Emotion       Id
0  My favourite food is anything I didn't have to...  neutral  eebbqej
1  Now if he does off himself, everyone will thin...  neutral  ed00q6i
2                     WHY THE FUCK IS BAYLESS ISOING    anger  eezlygj
3                        To make her feel threatened     fear  ed7ypvh
4                             Dirty Southern Wankers    anger  ed0bdzj
                                                Text  Emotion       Id
0  Is this in New Orleans?? I really feel like th...  neutral  edgurhb
1  You know the answer man, you are programmed to...      joy  ee84bjg
2               I've never been this sad in my life!  sadness  edcu99z
3  The economy is heavily controlled and subsidiz...      joy  edc32e2
4  He could have easily taken a real camera from ...      joy  eepig6r
                                                Text  Emotion       Id
0  I’m really sorry about your situation :( Altho...  sadness  eecwqtt
1    I

## Preprocessing pipeline

The following cells contain all the operations performed on the corpuses in order to fit the model with appropriate data 

In [ ]:
#dropping column "Id"
train_set = train_set.drop("Id", axis = 1)
val_set = val_set.drop("Id", axis = 1)
test_set = test_set.drop("Id", axis = 1)

print("Column dropped")

Column dropped


In [ ]:
# here a text preprocessor from ekphrasis is built, notice that there's no tokenizer
preprocessor = TextPreProcessor(
  omit=['email', 'percent', 'money', 'phone', 'user', 'time', 'date', 'hashtag'],
  unpack_contractions = True,
  unpack_hashtags = False,
  corrector="twitter" # similar tasks of sentiment analisys were performed on twitter
                      # so i decided to stick with this corrector
)

# encoding of the target labels
encoder = OneHotEncoder(sparse=False)
encoder.fit(train_set["Emotion"].to_numpy().reshape(-1, 1))
num_classes = len(pd.unique(train_set["Emotion"]))

# instancing the segmenter from ekphrasis, training without it have also been performed
# but the scores were lower
segmenter = Segmenter(corpus='english')

def clean_text(txt):
  x = replace_with_desc(txt, " ")           # change emojis with their description
  x = x.lower()
  x = re.sub("\[[^\]]*\]\s*",'',x)          # remove text between square brackets
  x = re.sub('http\S*', '', x)              # remove urls
  x = re.sub('’', '\'', x)                  # standardize apostrophe
  if '/r' in x:                             # check the presence of a subreddit name
    x = re.sub('r/', '', x)
    x = segmenter.segment(x)
  x = re.sub(r'[^a-zA-Z\'!?\.]+', ' ', x)                               # remove special char, keeping only letters and '
  x = preprocessor.pre_process_doc(x)                                   # remove eccess whitespace, unpack contraction
  return x

def preprocess(df):
  text = df["Text"]                                                     # isolate text
  text = text.apply(clean_text).to_numpy()                              # apply text cleaning
  classes = encoder.transform(df["Emotion"].to_numpy().reshape(-1, 1))  # encode target
  classes = classes[text.astype(bool)]                                  # remove empty strings
  text = text[text.astype(bool)]                                        # and their respective emotions
  return text, classes

# actual preprocess call
x_train, y_train = preprocess(train_set)
x_val, y_val = preprocess(val_set)
x_test, y_test = preprocess(test_set)

print("Training set, after preprocessing: ")
print(x_train)

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
You can't omit/backoff and unpack hashtags!
 unpack_hashtags will be set to False
Reading english - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_1grams.txt


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_2grams.txt
Training set, after preprocessing: 
['my favourite food is anything i did not have to cook myself.'
 'now if he does off himself everyone will think hes having a laugh screwing with people instead of actually dead'
 'why the fuck is bayless isoing' ...
 'what are you talking about? anything bad that happened was fault only good things were doing!'
 'more like a baptism with sexy results!' 'enjoy the ride!']


## Instantiation of the RoBERTa tokenizer

source: https://huggingface.co/docs/transformers/model_doc/roberta#transformers.RobertaTokenizer

This tokenizer is derived from the GPT-2 tokenizer, given a string or a list of , it returns 
a representation of it as a pair of vectors (input_ids, attention_mask), where:
- The input ids are token indices, numerical representations of tokens building the sequences that will be used as input by the model, the length of this vector depends on the length of the string
- The attention masks are vectors needed to keep track of where the padding indexes are kept in the input ids, in order to keep in the model the information of the values that need to be attended to, and the padded ones.

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base", model_max_length = 211, padding_side = "right")

x_train_list = list(x_train)#.to_numpy())
x_val_list = list(x_val)#.to_numpy())
x_test_list = list(x_test)#.to_numpy())

encoded_train = tokenizer(x_train_list, return_tensors = "tf", padding = True)
encoded_val = tokenizer(x_val_list, return_tensors = "tf", padding = True)
encoded_test = tokenizer(x_test_list, return_tensors = "tf", padding = True)

print("Training set, after the tokenization step")
print(encoded_train.keys)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Training set, after the tokenization step
<bound method BatchEncoding.keys of {'input_ids': <tf.Tensor: shape=(43396, 211), dtype=int32, numpy=
array([[    0,  4783,  5548, ...,     1,     1,     1],
       [    0,  8310,   114, ...,     1,     1,     1],
       [    0, 25800,     5, ...,     1,     1,     1],
       ...,
       [    0, 12196,    32, ...,     1,     1,     1],
       [    0,  4321,   101, ...,     1,     1,     1],
       [    0,   225, 20768, ...,     1,     1,     1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(43396, 211), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}>


In [ ]:
# Since the output of the tokenizer varies with the size of the input
# we need to resize the validation and test set accordingly
def resize_according_train(encoded_train, to_resize):
  enc_t_ids = encoded_train["input_ids"]
  enc_t_att = encoded_train["attention_mask"]
  to_resize_t_ids = np.array(to_resize["input_ids"])
  to_resize_t_att = np.array(to_resize["attention_mask"])
  to_resize_new_ids = np.ones((to_resize_t_ids.shape[0], enc_t_ids.shape[1]), dtype = np.int32)
  to_resize_new_att = np.zeros((to_resize_t_att.shape[0], enc_t_att.shape[1]), dtype = np.int32)
  to_resize_new_ids[:to_resize_t_ids.shape[0], :to_resize_t_ids.shape[1]] = to_resize_t_ids
  to_resize_new_att[:to_resize_t_att.shape[0], :to_resize_t_att.shape[1]] = to_resize_t_att
  to_return = {"input_ids":tf.convert_to_tensor(to_resize_new_ids, dtype = np.int32), "attention_mask":tf.convert_to_tensor(to_resize_new_att, dtype = np.int32)}
  return to_return

In [ ]:
encoded_val = resize_according_train(encoded_train, encoded_val)
encoded_test = resize_according_train(encoded_train, encoded_test)

print("Performed resize on the encoded validation test: ")
print(encoded_val["input_ids"])

Performed resize on the encoded validation test: 
tf.Tensor(
[[   0  354   42 ...    1    1    1]
 [   0 6968  216 ...    1    1    1]
 [   0  118   33 ...    1    1    1]
 ...
 [   0  627   94 ...    1    1    1]
 [   0  118   64 ...    1    1    1]
 [   0 4297  939 ...    1    1    1]], shape=(5425, 211), dtype=int32)


## The model

In this cell we define and compile the model in a function, in order to refresh
weigths each time we call the function (essentially because we have a new value assigned to the pointer "model")

In [ ]:
MAX_LEN = len(encoded_train["input_ids"][0])

def gpu():
  with tf.device('/device:GPU:0'):
    # the metrics that we want to monitor are just the loss and the F1Score, so we keep them
    metrics=[F1Score(num_classes=7)]

    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)                     # first layer of input of the network, input ids vectors
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)                     # second layer of input of the network, attention masks vectors

    roberta_model = TFRobertaModel.from_pretrained("roberta-base")              # importing the base roberta model
    roberta_model.trainable = False                                             # RoBERTa is a large model, no reason for it to be trained again, 
                                                                                # we should perform fine tuning instead, to do so we need to set the train to False
    x = roberta_model(ids,attention_mask=att)                                   # this will be the first layer of our complete model
    x.trainable = False                                                         
    bid_lstm = Bidirectional(LSTM(200, return_sequences=True))(x[0])            # first we need to add a Bi-LSTM layer, wich is used to keep track of contextual informations
                                                                                # of the words in the sentence. 
                                                                                # The choice of 200 as a parameter is because of the fact that the task and the dataset dimensions
                                                                                # of the paper and the one presented here, are similar. 

    x = SeqSelfAttention(400)(bid_lstm)                                         # The self attention layer is needed to weight the tokens with respect to the neighbouring ones
                                                                                # according to their similarity

    x = Convolution1D(400, 5)(x)                                                # Convolutional layer to capture hidden features
    x = MaxPooling1D(2)(x)                                                      # Subsampling of the values
    x = Dropout(0.3)(x)                                                         # Dropout in order to avoid overfitting 
    x = Concatenate(axis=1)([x, bid_lstm])                                      # Concatenation of the features extracted and the output of the LSTM, needed in order to
                                                                                # acquire relationships between past inputs
    x = GlobalMaxPool1D()(x)                                                    # last layers of the network, were we link our extracted features to the target classes
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(7, activation = 'softmax')(x)

    

    model = tf.keras.models.Model(inputs=[ids, att], outputs=[x])               
    # for classification tasks like this one (one hot encoded labels) it's necessary to use the categorical cross entropy as loss
    model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=metrics) 


  
  return model

model = gpu()
model.summary()

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None, 211)]        0           []                               
                                                                                                  
 input_18 (InputLayer)          [(None, 211)]        0           []                               
                                                                                                  
 tf_roberta_model_20 (TFRoberta  TFBaseModelOutputWi  124645632  ['input_17[0][0]',               
 Model)                         thPoolingAndCrossAt               'input_18[0][0]']               
                                tentions(last_hidde                                               
                                n_state=(None, 211,                                         

## Training Function

The parameters used to train the model have been chosen on the basis of various tries: 
- the number of epochs depend on how the model fits the data, thus, in order to find an appropriate value, it is possible to set epochs = 1, run the cell, inspect the results and choose to keep going on with training or stop because of overfitting (validation loss not improving and degraded performances on the validation set, while improving performances on the training set);
- the method to chose the batch size is more theoretical: as RoBERTa should be trained on large batches, we should follow the same guideline even during fine tuning, 32 can be considered as a quite large batch, at least in this context (smaller batch sizes have been tested: 26 and 13, but the best performances were obtained with 32).

It is possible to run the cell again, after the completion of a whole epoch, because <code>model</code> still references the same neural network, so running the <code>fit</code> method will result in a training step that starts from the last saved weights.

In [ ]:
sv = tf.keras.callbacks.ModelCheckpoint(
       "/content/Ammiraglio.h5", monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=True, mode='auto', save_freq='epoch') # The choice of this file name is related to my Admiral Mary Read
                                                                # whose real name is Roberta

model.fit([encoded_train["input_ids"], encoded_train["attention_mask"]], y_train, 
          batch_size=32, epochs=4, verbose = 1, callbacks = sv, 
          validation_data = ([encoded_val["input_ids"], encoded_val["attention_mask"]], y_val)
          )

<built-in method keys of dict object at 0x7f0b46182d70>
26
Epoch 1/4
1357/1357 [==============================] - ETA: 0s - loss: 1.1163 - f1_score: 0.4271
Epoch 1: val_loss improved from inf to 0.97916, saving model to /content/Ammiraglio.h5
1357/1357 [==============================] - 1069s 779ms/step - loss: 1.1163 - f1_score: 0.4271 - val_loss: 0.9792 - val_f1_score: 0.5755
Epoch 2/4
1357/1357 [==============================] - ETA: 0s - loss: 0.9577 - f1_score: 0.5524
Epoch 2: val_loss improved from 0.97916 to 0.92018, saving model to /content/Ammiraglio.h5
1357/1357 [==============================] - 1054s 777ms/step - loss: 0.9577 - f1_score: 0.5524 - val_loss: 0.9202 - val_f1_score: 0.5683
Epoch 3/4
1357/1357 [==============================] - ETA: 0s - loss: 0.9044 - f1_score: 0.5739
Epoch 3: val_loss improved from 0.92018 to 0.89634, saving model to /content/Ammiraglio.h5
1357/1357 [==============================] - 1054s 777ms/step - loss: 0.9044 - f1_score: 0.5739 - val_los

In [ ]:
# Saving the model, then loading it again and performing the predictions over the test set
model.save("/content/Ammiraglio_model_3.h5")
loaded_model = tf.keras.models.load_model("/content/Ammiraglio_model_3.h5", 
                                          custom_objects={"TFRobertaModel":TFRobertaModel.from_pretrained("roberta-base"),  
                                                          "SeqSelfAttention":SeqSelfAttention}, compile = False)
obtained_preds = loaded_model.predict([encoded_test["input_ids"], encoded_test["attention_mask"]])

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


## Best fit obtained so far

As shown in the papers, an F1 Macro score of 0.60 on an imbalanced dataset like this one is quite good, even tho for some applications RoBERTa shows better performances. 

In [ ]:
def keep_only_best_pred(obtained_preds):
  to_return = np.zeros(obtained_preds.shape, dtype = np.int32)
  for i in range(0, obtained_preds.shape[0]):
    to_return[i, np.argmax(obtained_preds[i, :])] = 1
  return to_return

y_pred = keep_only_best_pred(obtained_preds)
print(classification_report(y_test, y_pred))
print(f1_score(y_test, y_pred, average = "macro"))

              precision    recall  f1-score   support

           0       0.51      0.47      0.49       572
           1       0.62      0.36      0.46       116
           2       0.64      0.68      0.66        81
           3       0.83      0.77      0.80      1978
           4       0.61      0.69      0.64      1648
           5       0.64      0.49      0.56       355
           6       0.56      0.64      0.60       677

   micro avg       0.67      0.67      0.67      5427
   macro avg       0.63      0.59      0.60      5427
weighted avg       0.68      0.67      0.67      5427
 samples avg       0.67      0.67      0.67      5427

0.6009313253656848


## Try the model on a string:

In [ ]:
str_to_process = "I cannot believe that this works!"
enc_before_resizing = tokenizer([str_to_process])
encoded_string = resize_according_train(encoded_train, enc_before_resizing)
prediction_vector = loaded_model.predict([encoded_string["input_ids"], encoded_string["attention_mask"]])
to_decode = np.zeros(prediction_vector.shape, dtype = np.int32)
to_decode[0, np.argmax(prediction_vector)] = 1
prediction_label = encoder.inverse_transform(to_decode)
print("The computed label for: " + str(str_to_process) + "\nis: " + str(prediction_label[0]))

The computed label for: I cannot believe that this works!
is: ['surprise']


## Alternative, more simple fine tuning model:


In [ ]:
MAX_LEN = len(encoded_train["input_ids"][0])

def gpu_2():
  with tf.device('/device:GPU:0'):
    metrics=[F1Score(num_classes=7)]

    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    roberta_model = TFRobertaModel.from_pretrained("roberta-base")
    roberta_model.trainable = False
    x = roberta_model(ids,attention_mask=att)
    x.trainable = False
    x = Bidirectional(LSTM(256, return_sequences=False))(x[0])                  # Bi-LSTM layer with a bigger dimension
    x = Dropout(0.3)(x)                                                         # Dropout layer to avoid overfitting
    x = Dense(512, activation='relu')(x)                                        # Simple dense-dropout layer pipeline
    x = Dropout(0.3)(x)                                                         
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(7, activation = 'softmax')(x)

    model = tf.keras.models.Model(inputs=[ids, att], outputs=[x])
    model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=metrics)

  
  return model

model = gpu_2()
model.summary()

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 211)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 211)]        0           []                               
                                                                                                  
 tf_roberta_model_6 (TFRobertaM  TFBaseModelOutputWi  124645632  ['input_3[0][0]',                
 odel)                          thPoolingAndCrossAt               'input_4[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 211,                                         

In [ ]:
sv = tf.keras.callbacks.ModelCheckpoint(
       "/content/Ammiraglio_more_simple.h5", monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=True, mode='auto', save_freq='epoch')

model.fit([encoded_train["input_ids"], encoded_train["attention_mask"]], y_train, 
          batch_size=32, epochs=4, verbose = 1, callbacks = sv, 
          validation_data = ([encoded_val["input_ids"], encoded_val["attention_mask"]], y_val)
          )

Epoch 1/4
1357/1357 [==============================] - ETA: 0s - loss: 1.1959 - f1_score: 0.3310
Epoch 1: val_loss improved from inf to 1.00973, saving model to /content/Ammiraglio_more_simple.h5
1357/1357 [==============================] - 822s 594ms/step - loss: 1.1959 - f1_score: 0.3310 - val_loss: 1.0097 - val_f1_score: 0.4253
Epoch 2/4
1357/1357 [==============================] - ETA: 0s - loss: 1.0256 - f1_score: 0.4389
Epoch 2: val_loss improved from 1.00973 to 0.93473, saving model to /content/Ammiraglio_more_simple.h5
1357/1357 [==============================] - 804s 592ms/step - loss: 1.0256 - f1_score: 0.4389 - val_loss: 0.9347 - val_f1_score: 0.5451
Epoch 3/4
1357/1357 [==============================] - ETA: 0s - loss: 0.9619 - f1_score: 0.5171
Epoch 3: val_loss improved from 0.93473 to 0.91024, saving model to /content/Ammiraglio_more_simple.h5
1357/1357 [==============================] - 802s 591ms/step - loss: 0.9619 - f1_score: 0.5171 - val_loss: 0.9102 - val_f1_score: 

In [ ]:
save_path = "/content/Ammiraglio_model_alternative.h5"
model.save(save_path)
loaded_model = tf.keras.models.load_model(save_path, 
                                          custom_objects={"TFRobertaModel":TFRobertaModel.from_pretrained("roberta-base")}, compile = False)
obtained_preds_more_simple = loaded_model.predict([encoded_test["input_ids"], encoded_test["attention_mask"]])

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
y_pred = keep_only_best_pred(obtained_preds_more_simple)
print(classification_report(y_test, y_pred))
print(f1_score(y_test, y_pred, average = "macro"))

              precision    recall  f1-score   support

           0       0.49      0.51      0.50       572
           1       0.56      0.32      0.41       116
           2       0.61      0.63      0.62        81
           3       0.76      0.84      0.80      1978
           4       0.64      0.60      0.62      1648
           5       0.56      0.51      0.53       355
           6       0.60      0.55      0.58       677

   micro avg       0.66      0.66      0.66      5427
   macro avg       0.60      0.57      0.58      5427
weighted avg       0.66      0.66      0.66      5427
 samples avg       0.66      0.66      0.66      5427

0.5799797056939819


## Comparison of the models
The best model is more complex because it takes into account temporal dimensionality between sentences and spatial information between tokens, counting 2,712,408 trainable parameters.
The second one has a larger number of units for the Bi-LSTM layer (similar scores can be obtained by sequentially add Bi-LSTM layers with a fraction of the number of units presented), but the deep features are extracted just with Dense layers, so it's less sophisticated.
Apart from the tokenization method, the training corpus had longer sentences 
than the validation and test corpus, but even if the data was imbalanced, both networks performed quite well (0.58-0.60 F1 macro).